In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load in data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_test['Transported'] = False

In [ ]:
# Concat and check
df = pd.concat([df_train, df_test], sort=False)
df.shape[0] == df_train.shape[0] + df_test.shape[0]
df.head()

In [ ]:
# Drop name column (not needed for outcome)
df.drop(['Name'], axis=1, inplace=True)

In [ ]:
# Check for na values
df.isna().sum()

### Categorical na values

In [ ]:
# Fill categorical na values with mode
for col in ['HomePlanet', 'Destination', 'CryoSleep', 'VIP']:
    df[col] = df[col].fillna(df[col].mode()[0])
df.isna().sum()

In [ ]:
# Encode booleans
df['CryoSleep'] = df['CryoSleep'].map({
    True: 1, False: 0
})

df['VIP'] = df['VIP'].map({
    True: 1, False: 0
})

In [ ]:
# Split 'Cabin' into Deck, Num, and Side and fill na
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
df = df.drop(columns=['Cabin'])
df['Deck'] = df['Deck'].fillna('U')
df['Num'] = df['Num'].fillna(-1)
df['Side'] = df['Side'].fillna('U')
df.head()

In [ ]:
# Encoding
df = pd.get_dummies(df, columns=['HomePlanet', 'Destination', 'Deck', 'Side'])

In [ ]:
# Check for missed na values
df.isna().sum()

### Numberical na values

In [ ]:
for col in ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
# Check for missed na values
df.isna().sum()

### Feature Engineering